In [87]:
import dataclasses

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import lightgbm
from sklearn.linear_model import LassoCV, LinearRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [4]:
@dataclasses.dataclass
class Data:
    train: pd.DataFrame
    test: pd.DataFrame

In [130]:
FILES = ('discounts_train.parq.gzip', 'discounts_test.parq.gzip')

sns.set_style("darkgrid")
pd.options.display.max_columns = 100

In [146]:
data = Data(pd.read_parquet(FILES[0]), pd.read_parquet(FILES[1]))

In [147]:
data.train.head()

,spend_1m_baby,spend_2m_baby,spend_3m_baby,spend_4m_baby,spend_5m_baby,spend_6m_baby,spend_1m_clothes,spend_2m_clothes,spend_3m_clothes,spend_4m_clothes,spend_5m_clothes,spend_6m_clothes,spend_1m_health,spend_2m_health,spend_3m_health,spend_4m_health,spend_5m_health,spend_6m_health,spend_1m_pet,spend_2m_pet,spend_3m_pet,spend_4m_pet,spend_5m_pet,spend_6m_pet,spend_1m_groceries,spend_2m_groceries,spend_3m_groceries,spend_4m_groceries,spend_5m_groceries,spend_6m_groceries,spend_1m_eletronic,spend_2m_eletronic,spend_3m_eletronic,spend_4m_eletronic,spend_5m_eletronic,spend_6m_eletronic,sales,discount,profit,age,gender,cust_state,tenure,sales_prediction_bins,sales_prediction
0,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,45,15,30.0,55.0,10.0,160.0,0,0,0,0.0,0.0,0.0,3368.64,125.0,40.06,40,1.0,MA,6,8,1351.024765
1,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,10.0,0.0,0,0,0,0.0,0.0,0.0,2133.10,75.0,29.52,36,0.0,MG,10,4,1035.580387
2,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,25,30,35.0,0.0,20.0,50.0,0,0,0,0.0,0.0,0.0,2001.62,50.0,48.08,34,0.0,RJ,7,3,992.401825
3,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,55,135,120.0,10.0,40.0,130.0,0,0,0,0.0,0.0,0.0,1461.96,10.0,61.64,31,0.0,BA,7,1,919.720735
4,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,10.0,95.0,25.0,20.0,0,0,0,0.0,0.0,0.0,2743.72,100.0,34.44,32,1.0,PB,6,6,1176.485681


In [148]:
unused = ['sales', 'discount', 'profit', 'sales_prediction_bins', 'sales_prediction']
keep = data.train.columns.drop(unused)
X = data.train[keep].fillna(0)
y = data.train['sales']

In [149]:
cat_processor = ColumnTransformer(
    [
        ('ohe', OneHotEncoder(), ['cust_state'])
    ],
    remainder='passthrough'
)

pl = Pipeline([
    ('preprocessor', cat_processor),
    ('scaler', StandardScaler()),
    ('feature_selection', RFECV(estimator=LinearRegression()))
])

pl.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe', OneHotEncoder(),
                                                  ['cust_state'])])),
                ('scaler', StandardScaler()),
                ('feature_selection', RFECV(estimator=LinearRegression()))])

In [150]:
mask = pl.named_steps.feature_selection.get_support()
rfe_select = pl.named_steps.preprocessor.get_feature_names_out()[mask]

In [151]:
rfe_select

array(['ohe__cust_state_AC', 'ohe__cust_state_AL', 'ohe__cust_state_AM',
       'ohe__cust_state_AP', 'ohe__cust_state_BA', 'ohe__cust_state_CE',
       'ohe__cust_state_DF', 'ohe__cust_state_ES', 'ohe__cust_state_GO',
       'ohe__cust_state_MA', 'ohe__cust_state_MG', 'ohe__cust_state_MS',
       'ohe__cust_state_MT', 'ohe__cust_state_PA', 'ohe__cust_state_PB',
       'ohe__cust_state_PE', 'ohe__cust_state_PI', 'ohe__cust_state_PR',
       'ohe__cust_state_RJ', 'ohe__cust_state_RN', 'ohe__cust_state_RO',
       'ohe__cust_state_RR', 'ohe__cust_state_RS', 'ohe__cust_state_SC',
       'ohe__cust_state_SE', 'ohe__cust_state_SP', 'ohe__cust_state_TO',
       'remainder__spend_1m_baby', 'remainder__spend_2m_baby',
       'remainder__spend_3m_baby', 'remainder__spend_4m_baby',
       'remainder__spend_5m_baby', 'remainder__spend_6m_baby',
       'remainder__spend_1m_clothes', 'remainder__spend_2m_clothes',
       'remainder__spend_3m_clothes', 'remainder__spend_4m_clothes',
       'remainde

<hr>

In [152]:
cat_processor = ColumnTransformer(
    [
        ('ohe', OneHotEncoder(), ['cust_state'])
    ],
    remainder='passthrough'
)

pl = Pipeline([
    ('preprocessor', cat_processor),
    ('scaler', StandardScaler()),
    ('feature_selection', RFECV(estimator=LinearRegression())),
    ('estimator', lightgbm.LGBMRegressor()),
])

pl.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe', OneHotEncoder(),
                                                  ['cust_state'])])),
                ('scaler', StandardScaler()),
                ('feature_selection', RFECV(estimator=LinearRegression())),
                ('estimator', LGBMRegressor())])

In [154]:
pl.score(X, y)

0.8960578369435123

In [155]:
train_pred = pd.DataFrame(data.train).fillna(0)
train_pred['sales_prediction'] = pl.predict(train_pred[keep])
train_pred.head()

,spend_1m_baby,spend_2m_baby,spend_3m_baby,spend_4m_baby,spend_5m_baby,spend_6m_baby,spend_1m_clothes,spend_2m_clothes,spend_3m_clothes,spend_4m_clothes,spend_5m_clothes,spend_6m_clothes,spend_1m_health,spend_2m_health,spend_3m_health,spend_4m_health,spend_5m_health,spend_6m_health,spend_1m_pet,spend_2m_pet,spend_3m_pet,spend_4m_pet,spend_5m_pet,spend_6m_pet,spend_1m_groceries,spend_2m_groceries,spend_3m_groceries,spend_4m_groceries,spend_5m_groceries,spend_6m_groceries,spend_1m_eletronic,spend_2m_eletronic,spend_3m_eletronic,spend_4m_eletronic,spend_5m_eletronic,spend_6m_eletronic,sales,discount,profit,age,gender,cust_state,tenure,sales_prediction_bins,sales_prediction
0,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,45,15,30.0,55.0,10.0,160.0,0,0,0,0.0,0.0,0.0,3368.64,125.0,40.06,40,1.0,MA,6,8,3349.977201
1,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,10.0,0.0,0,0,0,0.0,0.0,0.0,2133.10,75.0,29.52,36,0.0,MG,10,4,2198.032312
2,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,25,30,35.0,0.0,20.0,50.0,0,0,0,0.0,0.0,0.0,2001.62,50.0,48.08,34,0.0,RJ,7,3,2135.131324
3,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,55,135,120.0,10.0,40.0,130.0,0,0,0,0.0,0.0,0.0,1461.96,10.0,61.64,31,0.0,BA,7,1,2245.074495
4,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,10.0,95.0,25.0,20.0,0,0,0,0.0,0.0,0.0,2743.72,100.0,34.44,32,1.0,PB,6,6,2137.711065


In [156]:
mean_absolute_error(train_pred['sales_prediction'], data.train['sales'])

264.5697262985526

In [157]:
test_pred = pd.DataFrame(data.test).fillna(0)
test_pred['sales_prediction'] = pl.predict(test_pred[keep])
test_pred.head()

,spend_1m_baby,spend_2m_baby,spend_3m_baby,spend_4m_baby,spend_5m_baby,spend_6m_baby,spend_1m_clothes,spend_2m_clothes,spend_3m_clothes,spend_4m_clothes,spend_5m_clothes,spend_6m_clothes,spend_1m_health,spend_2m_health,spend_3m_health,spend_4m_health,spend_5m_health,spend_6m_health,spend_1m_pet,spend_2m_pet,spend_3m_pet,spend_4m_pet,spend_5m_pet,spend_6m_pet,spend_1m_groceries,spend_2m_groceries,spend_3m_groceries,spend_4m_groceries,spend_5m_groceries,spend_6m_groceries,spend_1m_eletronic,spend_2m_eletronic,spend_3m_eletronic,spend_4m_eletronic,spend_5m_eletronic,spend_6m_eletronic,sales,discount,age,gender,cust_state,tenure,profit,sales_prediction
0,0,0,0,0,0,0.0,25,50.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,1503.22,20.0,35,1.0,MA,23,55.16,2292.466177
1,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,10,10,5.0,5.0,0.0,0.0,0,0,0,0.0,0.0,0.0,1798.30,60.0,29,1.0,MG,18,29.92,1734.958214
2,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,1122.97,0.0,30,1.0,RJ,20,56.15,1855.784629
3,10,15,0,25,10,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,15,25,95.0,10.0,0.0,0.0,0,0,0,0.0,0.0,0.0,1217.08,10.0,32,0.0,BA,20,50.85,2236.858599
4,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,966.58,0.0,30,0.0,PB,22,48.33,1667.244759


In [163]:
pl.score(test_pred[keep], test_pred.loc[:, 'sales_prediction'])

1.0

In [165]:
mean_absolute_error(test_pred['sales_prediction'], data.test['sales'])

609.023274245494

In [166]:
def elast(data, x, y):
    cov = data[[x, y]].cov()
    return cov.iloc[1, 0] / cov.iloc[0, 0]

In [167]:
sorted_by_pred_test = test_pred.sort_values('sales_prediction', ascending=False)
sorted_by_pred_test.head()

,spend_1m_baby,spend_2m_baby,spend_3m_baby,spend_4m_baby,spend_5m_baby,spend_6m_baby,spend_1m_clothes,spend_2m_clothes,spend_3m_clothes,spend_4m_clothes,spend_5m_clothes,spend_6m_clothes,spend_1m_health,spend_2m_health,spend_3m_health,spend_4m_health,spend_5m_health,spend_6m_health,spend_1m_pet,spend_2m_pet,spend_3m_pet,spend_4m_pet,spend_5m_pet,spend_6m_pet,spend_1m_groceries,spend_2m_groceries,spend_3m_groceries,spend_4m_groceries,spend_5m_groceries,spend_6m_groceries,spend_1m_eletronic,spend_2m_eletronic,spend_3m_eletronic,spend_4m_eletronic,spend_5m_eletronic,spend_6m_eletronic,sales,discount,age,gender,cust_state,tenure,profit,sales_prediction
2913,0,0,0,0,0,1500.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,20,10,160.0,150.0,10.0,170.0,500,500,1500,500.0,500.0,0.0,10795.12,150.0,61,1.0,SP,20,389.76,8197.088470
4683,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,15,15,0.0,10.0,25.0,5.0,500,500,2000,2000.0,0.0,0.0,8443.42,100.0,35,1.0,PR,24,322.17,6734.040833
1736,0,0,0,0,0,3000.0,0,0.0,0.0,0.0,900.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,300.0,300.0,300.0,300.0,0,0,0,0.0,0.0,0.0,11917.34,300.0,43,1.0,SC,19,295.87,6377.933209
1326,0,0,0,0,0,1900.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,10,5,195.0,195.0,15.0,200.0,0,0,0,0.0,0.0,0.0,6958.75,190.0,33,0.0,SP,22,157.94,6331.111181
3616,0,0,0,0,0,2300.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,35,25,275.0,245.0,275.0,275.0,0,0,0,0.0,0.0,0.0,7253.21,230.0,29,0.0,PI,21,132.66,6085.352801


In [168]:
elast(sorted_by_pred_test.iloc[:1000], 'discount', 'profit')

-0.09681784400126657

In [169]:
elast(sorted_by_pred_test.iloc[-1000:], 'discount', 'profit')

-0.2694978560937248

In [170]:
train_pred.to_parquet('discounts_train_scored.parq.gzip')
test_pred.to_parquet('discounts_test_scored.parq.gzip')